In [140]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# Dependencies
import numpy as np
import pandas as pd
import requests
import unidecode
import datetime
import dateutil
import subprocess
import sys
import json
import tempfile
import os
import re

# Install missing dependencies
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

---

# Colombia Covid19 Time Line
Dataset obtained from [Instituto Nacional de Salud](https://www.ins.gov.co/Noticias/Paginas/Coronavirus.aspx) daily report Covid19 from Colombia.

You can get the official dataset here: 
[INS - Official Report](https://www.datos.gov.co/Salud-y-Protecci-n-Social/Casos-positivos-de-COVID-19-en-Colombia/gt2j-8ykr)

The number of new cases are increasing day by day around the world.
This dataset has information about reported cases from 32 Colombia departments.

Also you can get the dataset Google COVID-19 Community Mobility Reports - Colombia.

You can view and collaborate to the analysis here:
[colombia_covid_19_analysis](https://www.kaggle.com/sebaxtian/colombia-covid-19-analysis) Kaggle Notebook Kernel.

---

## Data Sources

In [141]:
# Input data files are available in the "../input/" directory.
INPUT_DIR = './'
if os.path.split(os.path.abspath('.'))[-1] == 'src':
    INPUT_DIR = '../input'
# Output data files are available in the "../output/" directory.
OUTPUT_DIR = './'
if os.path.split(os.path.abspath('.'))[-1] == 'src':
    OUTPUT_DIR = '../output'

In [142]:
# Covid19 Colombia Dataset
covid19co = pd.read_csv(os.path.join(OUTPUT_DIR, 'covid19co.csv'))
# Total Covid19 Colombia
covid19co.shape

(7973, 46)

In [143]:
# Covid19 Colombia Samples Processed Dataset
covid19co_samples_processed = pd.read_csv(os.path.join(OUTPUT_DIR, 'covid19co_samples_processed.csv'))
# Total Covid19 Colombia Samples Processed
covid19co_samples_processed.shape

(62, 2)

---

## Official Date Report

In [144]:
# URL Bogota Date
URL_BOGOTA_TIME = 'http://worldtimeapi.org/api/timezone/America/Bogota'
# Get Bogota Date
with requests.get(URL_BOGOTA_TIME) as bogota_time:
    bogota_time = bogota_time.json()
# Bogota Date
#print(bogota_time)
bogota_date = datetime.date.fromtimestamp(bogota_time['unixtime']).isoformat()
print('Bogota Date:', bogota_date)

try:
    # URL Date Report
    URL_DATE_REPORT = 'https://e.infogram.com/api/live/flex/efcb7f88-4bd0-4e26-a497-14ae28f6d199/a90dbc02-108d-44be-8178-b1eb6ea1fdd9?'
    # Get Official Date Report
    with requests.get(URL_DATE_REPORT) as official_date_report:
        official_date_report = official_date_report.json()
    # Official Date Report
    #print(official_date_report['data'][0][1][0])
    official_date_report = official_date_report['data'][0][1][0]
    #print(official_date_report)
    # Date Format
    date_format = official_date_report.split(' ')[4].split('-')
    # YEAR-MONTH-DAY
    official_date_report = datetime.date(int(date_format[2]), int(date_format[1]), int(date_format[0]))
except:
    official_date_report = bogota_date
# Print
print('Official Date Report:', official_date_report)

Bogota Date: 2020-05-05
Official Date Report: 2020-05-04


---

## Time Line Reported, Recupered and Deceased

In [145]:
# Show dataframe
covid19co.tail()

,ID DE CASO,FECHA DE NOTIFICACION,CODIGO DIVIPOLA,CIUDAD DE UBICACION,DEPARTAMENTO O DISTRITO,ATENCION,EDAD,SEXO,TIPO,ESTADO,...,FECHA RECUPERADO DIA,FECHA RECUPERADO MES,FECHA RECUPERADO ANIO,FECHA RECUPERADO NOMBRE MES,FECHA RECUPERADO DIA SEMANA,FECHA REPORTE WEB DIA,FECHA REPORTE WEB MES,FECHA REPORTE WEB ANIO,FECHA REPORTE WEB NOMBRE MES,FECHA REPORTE WEB DIA SEMANA
7968,8009,01/05/2020,15835,Turmequé,Boyacá,Casa,42,M,En Estudio,Leve,...,-,-,-,-,-,4,5,2020,Mayo,Lunes
7969,8010,01/05/2020,15001,Tunja,Boyacá,Casa,44,M,Relacionado,Leve,...,-,-,-,-,-,4,5,2020,Mayo,Lunes
7970,8011,30/04/2020,76001,Cali,Valle Del Cauca,Casa,28,F,En Estudio,Leve,...,-,-,-,-,-,4,5,2020,Mayo,Lunes
7971,8012,01/05/2020,25754,Soacha,Cundinamarca,Casa,49,F,En Estudio,Leve,...,-,-,-,-,-,4,5,2020,Mayo,Lunes
7972,8013,02/05/2020,25286,Funza,Cundinamarca,Casa,38,M,En Estudio,Leve,...,-,-,-,-,-,4,5,2020,Mayo,Lunes


In [146]:
# Get Time Line
def get_time_line(dfreport):
    # Time Line [date, total, accum]
    dfreport_time_line = pd.DataFrame(columns=['date', 'total', 'accum'])
    dfreport_time_line['date'] = [dti.strftime('%d/%m/%Y') for dti in pd.date_range(start='2020-03-01', end=official_date_report, freq='D')]
    # Total by Date
    total_by_date = {}
    # Group by 'FECHA REPORTE WEB'
    group_by_date = dfreport.groupby(['FECHA REPORTE WEB'], sort=False)
    # For each date
    for date_report in group_by_date.groups.keys():
        total_by_date[date_report] = group_by_date.get_group(date_report)['ID DE CASO'].count()
    # Update Total by Date
    dfreport_time_line['total'] = dfreport_time_line['date'].transform(lambda date: total_by_date[date] if date in total_by_date else 0)
    # Update Accumulative Sum Cases Reported by Date
    dfreport_time_line['accum'] = dfreport_time_line['total'].cumsum()
    # Drop the last one if doesn't have total
    #index_empty = dfreport_time_line[dfreport_time_line['date'] == datetime.date.today().strftime('%d/%m/%Y')]
    #index_empty = index_empty[index_empty['total'] == 0].index
    #dfreport_time_line.drop(index_empty, inplace=True)
    # Return
    return dfreport_time_line

In [147]:
# Get Reported Time Line
reported_time_line = get_time_line(covid19co)
# Rename columns
reported_time_line.columns = ['date', 'total_reported', 'accum_reported']
# Show dataframe
reported_time_line.tail()

,date,total_reported,accum_reported
60,30/04/2020,296,6466
61,01/05/2020,500,6966
62,02/05/2020,279,7245
63,03/05/2020,423,7668
64,04/05/2020,305,7973


In [148]:
# Get Recupered Time Line
dfrecupered = covid19co[covid19co['ATENCION'] == 'Recuperado']
# Get Recupered Time Line
recupered_time_line = get_time_line(dfrecupered)
# Rename columns
recupered_time_line.columns = ['date_recupered', 'total_recupered', 'accum_recupered']
# Show dataframe
recupered_time_line.tail()

,date_recupered,total_recupered,accum_recupered
60,30/04/2020,0,1807
61,01/05/2020,0,1807
62,02/05/2020,0,1807
63,03/05/2020,0,1807
64,04/05/2020,0,1807


In [149]:
# Get Deceased Time Line
dfdeceased = covid19co[covid19co['ATENCION'] == 'Fallecido']
# Get Deceased Time Line
deceased_time_line = get_time_line(dfdeceased)
# Rename columns
deceased_time_line.columns = ['date_deceased', 'total_deceased', 'accum_deceased']
# Show dataframe
deceased_time_line.tail()

,date_deceased,total_deceased,accum_deceased
60,30/04/2020,15,326
61,01/05/2020,10,336
62,02/05/2020,10,346
63,03/05/2020,8,354
64,04/05/2020,4,358


In [150]:
# Merge Time Lines
covid19co_time_line = pd.concat([reported_time_line, recupered_time_line, deceased_time_line], axis=1, sort=False)
# Delete Columns
covid19co_time_line.drop(columns=['date_recupered', 'date_deceased'], inplace=True)
# Show dataframe
covid19co_time_line.tail()

,date,total_reported,accum_reported,total_recupered,accum_recupered,total_deceased,accum_deceased
60,30/04/2020,296,6466,0,1807,15,326
61,01/05/2020,500,6966,0,1807,10,336
62,02/05/2020,279,7245,0,1807,10,346
63,03/05/2020,423,7668,0,1807,8,354
64,04/05/2020,305,7973,0,1807,4,358


## Time Line Reported, Recupered and Deceased
> ***Output file***: covid19co_time_line.csv

In [151]:
# Save dataframe
covid19co_time_line.to_csv(os.path.join(OUTPUT_DIR, 'covid19co_time_line.csv'), index=False)

---

## Time Line Reported, Recupered and Deceased by City

In [152]:
# List of Cities
cities = list(set(covid19co['CIUDAD DE UBICACION'].values))
# Show total cities
len(cities)

233

In [153]:
# Time Line by City
time_line_by_city = {}
# For each city
for city in cities:
    # Filter by City
    covid19co_city = covid19co[covid19co['CIUDAD DE UBICACION'] == city]
    # Get Reported Time Line
    reported_time_line = get_time_line(covid19co_city)
    # Rename columns
    reported_time_line.columns = ['date', 'total_reported', 'accum_reported']
    # Get Recupered Time Line
    dfrecupered = covid19co_city[covid19co_city['ATENCION'] == 'Recuperado']
    # Get Recupered Time Line
    recupered_time_line = get_time_line(dfrecupered)
    # Rename columns
    recupered_time_line.columns = ['date_recupered', 'total_recupered', 'accum_recupered']
    # Get Deceased Time Line
    dfdeceased = covid19co_city[covid19co_city['ATENCION'] == 'Fallecido']
    # Get Deceased Time Line
    deceased_time_line = get_time_line(dfdeceased)
    # Rename columns
    deceased_time_line.columns = ['date_deceased', 'total_deceased', 'accum_deceased']
    # Merge Time Lines
    covid19co_time_line = pd.concat([reported_time_line, recupered_time_line, deceased_time_line], axis=1, sort=False)
    # Delete Columns
    covid19co_time_line.drop(columns=['date_recupered', 'date_deceased'], inplace=True)
    # Create key city
    key_city = ''.join(x for x in re.sub('[^A-Za-z0-9 ]+', '', unidecode.unidecode(city)).title() if not x.isspace())
    # Add to dict
    time_line_by_city[key_city] = covid19co_time_line

In [154]:
# Show time line by city keys
#list(time_line_by_city.keys())

In [155]:
# Show dataframe
time_line_by_city['Cali'].tail()

,date,total_reported,accum_reported,total_recupered,accum_recupered,total_deceased,accum_deceased
60,30/04/2020,37,750,0,194,1,49
61,01/05/2020,48,798,0,194,1,50
62,02/05/2020,41,839,0,194,0,50
63,03/05/2020,2,841,0,194,0,50
64,04/05/2020,26,867,0,194,0,50


## Time Line Reported, Recupered and Deceased by City
> ***Output file***: covid19co_time_line_{key_city}.csv

In [156]:
for key_city in time_line_by_city:
    # Save dataframe
    time_line_by_city[key_city].to_csv(os.path.join(OUTPUT_DIR, 'covid19co_time_line_' + key_city + '.csv'), index=False)

---

## Time Line Reported, Recupered and Deceased by Department

In [157]:
# List of Departments
departs = list(set(covid19co['DEPARTAMENTO O DISTRITO '].values))
# Show total departments
len(departs)

31

In [158]:
# Time Line by Department
time_line_by_depto = {}
# For each deparment
for deparment in departs:
    # Filter by Department
    covid19co_depto = covid19co[covid19co['DEPARTAMENTO O DISTRITO '] == deparment]
    # Get Reported Time Line
    reported_time_line = get_time_line(covid19co_depto)
    # Rename columns
    reported_time_line.columns = ['date', 'total_reported', 'accum_reported']
    # Get Recupered Time Line
    dfrecupered = covid19co_depto[covid19co_depto['ATENCION'] == 'Recuperado']
    # Get Recupered Time Line
    recupered_time_line = get_time_line(dfrecupered)
    # Rename columns
    recupered_time_line.columns = ['date_recupered', 'total_recupered', 'accum_recupered']
    # Get Deceased Time Line
    dfdeceased = covid19co_depto[covid19co_depto['ATENCION'] == 'Fallecido']
    # Get Deceased Time Line
    deceased_time_line = get_time_line(dfdeceased)
    # Rename columns
    deceased_time_line.columns = ['date_deceased', 'total_deceased', 'accum_deceased']
    # Merge Time Lines
    covid19co_time_line = pd.concat([reported_time_line, recupered_time_line, deceased_time_line], axis=1, sort=False)
    # Delete Columns
    covid19co_time_line.drop(columns=['date_recupered', 'date_deceased'], inplace=True)
    # Create key depto
    key_depto = ''.join(x for x in re.sub('[^A-Za-z0-9 ]+', '', unidecode.unidecode(deparment)).title() if not x.isspace())
    # Add to dict
    time_line_by_depto[key_depto] = covid19co_time_line

In [159]:
# Show time line by deparment keys
#list(time_line_by_depto.keys())

In [160]:
# Show dataframe
time_line_by_depto['ValleDelCauca'].tail()

,date,total_reported,accum_reported,total_recupered,accum_recupered,total_deceased,accum_deceased
60,30/04/2020,39,917,0,250,2,61
61,01/05/2020,63,980,0,250,2,63
62,02/05/2020,42,1022,0,250,0,63
63,03/05/2020,2,1024,0,250,0,63
64,04/05/2020,34,1058,0,250,0,63


## Time Line Reported, Recupered and Deceased by Department
> ***Output file***: covid19co_time_line_{key_depto}.csv

In [161]:
for key_depto in time_line_by_depto:
    # Save dataframe
    time_line_by_depto[key_depto].to_csv(os.path.join(OUTPUT_DIR, 'covid19co_time_line_' + key_depto + '.csv'), index=False)

---

## Time Line Samples Processed

In [162]:
# Show dataframe
covid19co_samples_processed.head()

,FECHA,MUESTRAS PROCESADAS POR CORTE
0,-,601
1,05/03/2020,636
2,06/03/2020,739
3,07/03/2020,810
4,08/03/2020,813


In [163]:
# Rename columns
covid19co_samples_processed.columns = ['date', 'accum_samples']
# Fill NaN
covid19co_samples_processed['accum_samples'].fillna(0, inplace=True)
# Update column type
covid19co_samples_processed['accum_samples'] = covid19co_samples_processed['accum_samples'].astype('int64')
# Show dataframe
covid19co_samples_processed.head()

,date,accum_samples
0,-,601
1,05/03/2020,636
2,06/03/2020,739
3,07/03/2020,810
4,08/03/2020,813


In [164]:
# Time Line [date, accum]
covid19co_samples_time_line = pd.DataFrame(columns=['date', 'accum'])
covid19co_samples_time_line['date'] = [dti.strftime('%d/%m/%Y') for dti in pd.date_range(start='2020-03-01', end=official_date_report, freq='D')]
# Get Accumulative Samples
def get_accum(date_sample):
    accum = covid19co_samples_processed[covid19co_samples_processed['date'] == date_sample]['accum_samples'].values
    return accum[0] if len(accum) > 0 else 0
# Update accum
covid19co_samples_time_line['accum'] = covid19co_samples_time_line['date'].transform(lambda value: get_accum(value))
# Add samples without date
#covid19co_samples_time_line.iloc[2] = list(covid19co_samples_processed.iloc[0])
# Show dataframe
covid19co_samples_time_line.tail()

,date,accum
60,30/04/2020,104657
61,01/05/2020,108950
62,02/05/2020,114602
63,03/05/2020,118801
64,04/05/2020,123029


## Time Line Samples Processed
> ***Output file***: covid19co_samples_time_line.csv

In [165]:
# Save dataframe
covid19co_samples_time_line.to_csv(os.path.join(OUTPUT_DIR, 'covid19co_samples_time_line.csv'), index=False)

---